# Forking out command line MPI tasks
If we want to fork out MPI processes on the nodes, we can do that via the use of `mpi_wrap()` and then Dask manages the srun task that launches the job. `mpi_wrap()` returns a dictionary:
```python
{"cmd": cmd_launched, "out": stdout_output, "err": stderr_output}
```
This (currently) requires a little boilerplate code to work as one might expect (see the example below).
## Interacting with the task
If you need to grab information from the executed task, you can either do something to parse this dictionary, or interact with the executed task via the file system (i.e., read a result file).

In [ ]:
import os
from jobqueue_features.clusters import CustomSLURMCluster
from jobqueue_features.decorators import on_cluster, mpi_task
from jobqueue_features.mpi_wrapper import mpi_wrap
from jobqueue_features.functions import set_default_cluster

In [ ]:
set_default_cluster(CustomSLURMCluster)

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="mpiCluster", walltime="00:04:00", nodes=2, mpi_mode=True, fork_mpi=True, queue='devel'
)

In [ ]:
custom_cluster

We need boilerplate code because our decorators insert some kwargs (that define things like the number of processes) that are required for `mpi_wrap` to execute the task

In [ ]:
@on_cluster(cluster=custom_cluster, cluster_id=custom_cluster.name)
@mpi_task(cluster_id=custom_cluster.name)
def mpi_wrap_task(**kwargs):
    return mpi_wrap(**kwargs)

In [ ]:
def forked_mpi():
    script_path = os.path.join(os.environ.get("JOBQUEUE_FEATURES_EXAMPLES"), "resources", "helloworld.py")
    t = mpi_wrap_task(executable="python", exec_args=script_path)
    print("Ran\n\t", t.result()["cmd"])
    result = t.result()["out"]
    # Need to decode the output string so it is easily printed
    return result.decode('UTF-8')

In [ ]:
result = forked_mpi()

In [ ]:
print(result)